In [71]:
import common
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import os, pathlib, enum
import json
from pathlib import Path
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError
import psycopg2

root_url = Path("https://mcfp.felk.cvut.cz/publicDatasets/")
directory_path = Path(f"mcfp/publicDataset/")

re_name = re.compile(r"name:\s+([\w\.\-\_]*)\b", re.MULTILINE | re.IGNORECASE)
re_li_md5 = re.compile(r"md5:\s+\b([A-Fa-f0-9]{32})\b$", re.MULTILINE | re.IGNORECASE)
re_li_sha256 = re.compile(r"sha256:\s+\b([A-Fa-f0-9]{64})\b$", re.MULTILINE | re.IGNORECASE)
re_md5 = re.compile(r"([a-f0-9]{32})", re.MULTILINE | re.IGNORECASE)
re_sha256 = re.compile(r"([a-f0-9]{64})", re.MULTILINE | re.IGNORECASE)

def re_run(self, rgx, string):
    match = rgx.search(string)
    return match.group(1) if match else None

def scrape_table(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    table = soup.find('table')
    if table is None:
        return None
    table_data = []
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        row_data = [col.text.strip() for col in columns]
        if row_data:
            table_data.append(row_data)
    return table_data

class File:
    def __init__(self, folder, name: str, size: str = "", ts: str = ""):
        self.folder = folder
        self.name = name
        self.ts = ts
        self.size = size
        self._content = None
        pass
    
    @property
    def content(self):
        if self._content:
            return self._content
        fpath = self.folder.dir.joinpath(self.name if len(self.name) else "index.html")
        url = self.folder.url.joinpath(self.name)
        if fpath.is_file():
            with open(fpath, "br") as fp:
                self._content = fp.read()
        else:
            response = requests.get(url)
            if response.status_code == 200:
                self.content = response.content
                with open(fpath, "bw") as fp:
                    fp.write(response.content)
        return self._content

    @property
    def txt(self):
        return self.content.decode("utf-8")
    
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name
                

class Readme(File):
    def __init__(self, folder, name: str, size: str = "", ts: str = ""):
        super().__init__(folder, name, size, ts)
        pass

    @property
    def lists(self):
        soup = BeautifulSoup(self.txt, 'html.parser')
        return [ li.text for li in soup.find_all('li') ]

    @property
    def sha256(self):
        match = re_li_sha256.search(self.txt)
        if match:
            return match.group(1)
        match = re_sha256.search(self.txt)
        if match:
            return match.group(1)
        # print("SHA256 not present.")
        return None
    
    @property
    def md5(self):
        match = re_li_md5.search(self.txt)
        if match:
            return match.group(1)
        match = re_md5.search(self.txt)
        if match:
            return match.group(1)
        # print("SHA256 not present.")
        return None

    @property
    def mwname(self):
        match = re_name.search(self.txt.lower())
        if match:
            return match.group(1)
        # print("Name not present.")
        return None

class MCFPFolderType(enum.Enum):
    BOTNET = 0
    MIXED = 1
    NORMAL = 2
    OTHER = 3
    pass

    @staticmethod
    def translate(name):
        if 0 <= name.find("Botnet"): return MCFPFolderType.BOTNET
        if 0 <= name.find("Mixed"): return MCFPFolderType.MIXED
        if 0 <= name.find("Normal"): return MCFPFolderType.NORMAL
        return MCFPFolderType.OTHER

class MCFPFolder:
    def __init__(self, name):
        self.name = name
        self.url = root_url.joinpath(name)
        self.dir = directory_path.joinpath(name)
        self.dir.mkdir(exist_ok=True)
        self.htmlindex = File(self, "")
        self.type = MCFPFolderType.translate(self.name)
        if self.type == MCFPFolderType.BOTNET:
            self.id = self.name.replace("CTU-Malware-Capture-Botnet-", "").replace("/", "")
        elif self.type == MCFPFolderType.NORMAL:
            self.id = self.name.replace("CTU-Malware-Capture-Normal-", "").replace("/", "")
        elif self.type == MCFPFolderType.MIXED:
            self.id = self.name.replace("CTU-Mixed-Capture-", "").replace("/", "")
        else:
            self.id = self.name
        pass

    @property
    def table(self):
        if not self.htmlindex.txt:
                raise Exception("No htmlindex for %s" % self.name)
        table = scrape_table(self.htmlindex.txt)
        if not table:
            raise Exception("No table in htmlindex for %s" % self.name)
        return [ row for row in table ]

    @property
    def files(self):
        return [ File(self, row[1], ts=row[2], size=row[3]) for row in self.table if row[1].lower().find("parent") == -1]

    @property
    def readmes(self):
        return [ file for file in self.files if file.name.lower().find("readme") >= 0 ]
        
    @property
    def readme(self):
        if self.readmes:
            tmp = [ file for file in self.readmes if file.name.lower() == "readme.html" ]
            file = tmp[0] if tmp else self.readmes[0]
            return Readme(self, file.name, ts=file.ts, size=file.size)
        return None

    @property
    def pcaps(self):
        return [ file for file in self.files if file.name.lower().endswith(".pcap") ]

    def __str__(self):
        return self.name
        



In [ ]:
pcaps = pd.read_sql("SELECT * FROM pcap WHERE infected is true", database.engine)

pcaps["mcfp_id"] = pcaps[pcaps["dataset"] == "CTU-13"]["name"].apply(lambda x: x.split("_")[0])
pcaps["mcfp_pcap"] = pcaps[pcaps["dataset"] == "CTU-13"]["name"].apply(lambda x: x[x.find("_"):])
ids = pcaps.set_index("mcfp_id").copy()
ids["find"] = False
ids["mwname"] = None
ids["sha256"] = None
ids["md5"] = None
ids["type"] = None
botnets = {}
for walker in directory_path.walk():
    for folder in walker[1]:
        folder = MCFPFolder(folder)
        # if folder.type in [ MCFPFolderType.BOTNET, MCFPFolderType.MIXED ] and folder.readme and (not folder.readme.sha256 and not folder.readme.md5):
            # print("%s\n\tmwname: %s\n\tsha256: %s\n\t   md5: %s\n\n" % (folder, folder.readme.mwname, folder.readme.sha256, folder.readme.md5))
        if folder.id in ids.index and folder.type in [ MCFPFolderType.BOTNET, MCFPFolderType.MIXED ]:
            try:
                ids.at[folder.id, "find"] = True
                ids.at[folder.id, "mwname"] = folder.readme.mwname
                ids.at[folder.id, "sha256"] = folder.readme.sha256
                ids.at[folder.id, "md5"] = folder.readme.md5
                ids.at[folder.id, "type"] = folder.type
            except Exception as e:
                if MCFPFolderType.MIXED == folder.type:
                    print("err", folder, e)
                    pass
                pass
            pass
        

pcap_mw = ids.groupby(["md5"]).agg({
    "name": lambda s: s.to_list(),
    "id": lambda s: s.to_list(),
    "mwname": lambda s: list(set(s.to_list())),
    "sha256": lambda s: list(set(s.to_list())),
    "type": lambda s: list(set(s.to_list())),
    "find": "sum"
})

pcap_mw["mwname"] = pcap_mw["mwname"].apply(lambda n: n[0] if n[0] else None)
pcap_mw["sha256"] = pcap_mw["sha256"].apply(lambda n: n[0] if n[0] else None)

pcap_mw = pcap_mw.reset_index()

pcap_mw


In [ ]:

for _, row in pcap_mw.iterrows():
    with database.engine.connect() as conn:
        try:
            bo = conn.execute(
                text("""
                INSERT INTO public.malware(
                name, sha256, md5)
                VALUES (:name, :sha256, :md5)
                RETURNING ID;
                """),
                [{"name": row["mwname"], "sha256": row["sha256"], "md5": row["md5"] }]
            )
            conn.commit()
        except IntegrityError as e:
            try:
                raise e.orig
            except psycopg2.errors.UniqueViolation as e:
                pass
            except:
                print(e)

db_malwares = pd.read_sql("""SELECT * FROM malware""", database.engine.connect()).set_index("md5")
for _, row in pcap_mw.iterrows():
    with database.engine.connect() as conn:
        mwid = int(db_malwares.loc[row["md5"], "id"])
        for pcapid in row["id"]:
            try:
                conn.execute(
                    text("""
                    UPDATE PCAP SET MALWARE_ID=:mwid WHERE id=:id;
                    """),
                    [{"id": int(pcapid), "mwid": mwid }]
                )
                conn.commit()
            except IntegrityError as e:
                try:
                    raise e.orig
                except psycopg2.errors.UniqueViolation as e:
                    pass
                except:
                    print(e)


In [ ]:

malwares = pd.read_sql("""SELECT * FROM malware""", database.engine.connect()).set_index("id")
pcaps = pd.read_sql("""SELECT * FROM PCAP WHERE dataset='CTU-13' and infected is true""", database.engine.connect())[["name", "malware_id", "id"]]
test = []
for _,pcap in pcaps.iterrows():
    md5 = malwares.loc[pcap["malware_id"], "md5"]
    mcfp_mw = pcap_mw.set_index("md5").loc[md5]
    test.append(pcap["id"] in mcfp_mw["id"])

all(test)

In [89]:
pcaps = pd.read_sql("SELECT * FROM pcap", database.engine)

for _, row in pcaps.iterrows():
    if row["mcfp_info"]:
        mcfp_info = row["mcfp_info"][1:-1].split(",")
        folder = MCFPFolder(mcfp_info[0])
        # if not (mcfp_info[3].replace("fixed.", "").replace("original.", "") in [ pcap.name for pcap in folder.pcaps ]):
        if not (mcfp_info[3] in [ pcap.name for pcap in folder.pcaps ]):
            print(row["id"])#, folder.id, folder.type, mcfp_info[3], folder.pcaps)